<a href="https://colab.research.google.com/github/david-levin11/Verification_Notebooks/blob/main/Alaska_Sounding_Archive_Plotter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Alaska Observed & Forecast Archive Sounding Plotter**
**Description--This script will download archive sounding data from the below sources**


*   The University of Wyoming RAOB archive [1973-present, depending on station]
*   Iowa State University's RAOB archive [1945-present, depending on station]
*   Iowa State University's BUFKIT archive [2011-present, depending on station & model]


Script development by David Levin, Arctic Testbed & Proving Ground, Anchorage Alaska (adapted from Steven Fleegel)

-Python package created by [SounderPy](https://github.com/kylejgillett/sounderpy/blob/main/README.md), the vertical profile data retrieval and analysis tool for Python
LATEST VERSION: v3.0.4 | RELEASED: June, 2024 | COPYRIGHT Kyle J Gillett, 2023, 2024

##**2 - Install and Import Packages**
This will take about a minute to run.  **You will only have to run this cell one time per session!**

In [ ]:
# @title
!pip install -q sounderpy
#!pip install airportsdata
#import airportsdata
import sounderpy as spy
from datetime import datetime, timedelta

##**3 - Sounding Options**

In [ ]:
#@markdown # **Single Sounding Options**

#@markdown ---
#@markdown ### Are we plotting observed (Obs) or model (Bufkit) soundings?:
source = "Obs" #@param ["Bufkit", "Obs"]
ob_source = "RAOB"
#@markdown #### **What Site Are We Plotting?**
#@markdown ###### Provide a 4 letter site ID for the Obs/Bufkit Location (has to be either an observed or bufkit site!).
site = "PANC" #@param {type:"string"}
#@markdown ---
#@markdown ### **Select your date/time (If you're using Bufkit data this will be your model run time)**
date = "2022-11-10" #@param {type:"date"}
hour = 0 #@param {type:"slider", min:0, max:23, step:3}

#@markdown ---
#@markdown ### **If you're using Bufkit data, what model are we plotting? (Note that not all models are available at all times for every site in AK)**
#@markdown ### The most reliable models for AK are the NAM and GFS.  If you have a local archive of Bufkit or model data you would like me to plot with this script, please reach out!
#@markdown **UPDATE!** As of 09/11/2024 the Bufkit Warehouse has been updated to run NamNest and HRRR profiles for the state of Alaska.  Many thanks to Daryl Herzmann for making this happen.
model = "NAM" #@param ["GFS", "NAM", "NAMNEST", "RAP", "HRRR", "SREF", "HIRESW"]
fcst_start = 0 #@param {type:"slider", min:0, max:23, step:1}
fcst_end = 6 #@param {type:"slider", min:0, max:23, step:1}
model_run_hours = [0,6,12,18]
#@markdown # **Sounding Customizations**
#@markdown ---
#@markdown ### Dark Mode?
Dark_Mode_Plot = False #@param {type:"boolean"}
#@markdown ### Full or simple sounding?
style = "full" #@param ["full", "simple"]
#@markdown ### What Storm Motion do you want to use?
Storm_Motion = "right_moving" #@param ["right_moving", "left_moving", "mean_wind"]
#Custom_SM_Dir = 180.0 # @param {type:"number"}
#Custom_SM_Spd = 25.0 # @param {type:"number"}
#if Storm_Motion == "custom":
#  Storm_Motion_SPY = [Custom_SM_Dir, Custom_SM_Spd]
#else:
#  Storm_Motion_SPY = Storm_Motion
#@markdown ### Do you want to modify the surface parcel?  If so, check the box and enter surface temp and dewpoint (deg F)?
Custom_T_Td = False #@param {type:"boolean"}
Custom_Sfc_T = 35 # @param {type:"number"}
Custom_Sfc_Td = 30 # @param {type:"number"}
if Custom_T_Td:
  Modified_T_Td = [((Custom_Sfc_T - 32) * 5 / 9), ((Custom_Sfc_Td - 32) * 5 / 9)]
else:
  Modified_T_Td = Custom_T_Td
#@markdown ---
#@markdown # **Composite Sounding Options**
#@markdown ### Check this box to plot a composite sounding
Composite_Sounding = True #@param {type:"boolean"}
#@markdown ---
#@markdown Enter the site for which you'd like a composite sounding
composite_site = "PANC" #@param {type:"string"}
#@markdown ### You can compare RAOB to RAOB, model to model at a single site, or model to RAOB at a single site
#@markdown Enter a list of dates in "YYYY-mm-dd-hh" (UTC) format(EX: "2024-06-06-12,2024-06-07-12,2024-06-08-12").  Entering one date here will allow you to compare multiple models at a single time.
dates = "2022-11-10-00" #@param {type:"string"}
# Convert the input string into a list of dates
dates_list = [dates.strip() for date in dates.split(",")]
#@markdown Enter your list of models or enter RAOB to compare a series of observed soundings.  If you'd like to compare models to RAOBs, simply include "RAOB" in your list.
#@markdown (EX: "NAM,GFS,RAOB" or "NAM,GFS,NAMNEST")
models = "NAM,GFS,RAOB" #@param {type:"string"}
model_list = [model.strip() for model in models.split(",")]
#@markdown Enter the forecast hour(s) you want to compare. (EX: 3,6,9 would overlay the 3, 6, and 9 hour forecasts from each model in your model list at each valid date/time in your datetime list)
hours = "12" #@param {type:"string"}
hour_list = [hour.strip() for hour in hours.split(",")]
#@markdown ### Dark Mode?
Dark_Mode_Composite = False #@param {type:"boolean"}
########################## Input adjustments & error checks ####################
def adjust_time(date_str, time):
    # Convert the date string to a datetime object
    date = datetime.strptime(date_str, "%Y-%m-%d")

    # Determine if the time is closer to 00:00 or 12:00
    if time < 6:  # Closer to midnight (00:00)
        return date.strftime("%m/%d/%Y"), 0
    elif time < 18:  # Closer to noon (12:00)
        return date.strftime("%m/%d/%Y"), 12
    else:  # Closer to next day's midnight (00:00)
        date += timedelta(days=1)  # Move to the next day
        return date.strftime("%m/%d/%Y"), 0

def adjust_time_composite(date_str, format="%Y-%m-%d-%H"):
    # Convert the date string to a datetime object
    date = datetime.strptime(date_str, format)
    hour = int(date.hour)
    # Determine if the time is closer to 00:00 or 12:00
    if hour < 6:  # Closer to midnight (00:00)
        return date.strftime("%m/%d/%Y"), 0
    elif hour < 18:  # Closer to noon (12:00)
        return date.strftime("%m/%d/%Y"), 12
    else:  # Closer to next day's midnight (00:00)
        date += timedelta(days=1)  # Move to the next day
        return date.strftime("%m/%d/%Y"), 0

# Function to format the date and hour
def format_date_and_hour(date_str, hour, composite=False):
    # Convert the date string to a datetime object
    if not composite:
      try:
        date = datetime.strptime(date_str, "%Y-%m-%d")
      except:
        date = datetime.strptime(date_str, "%m/%d/%Y")
      # Extract year, month, day, and format them with leading zeros
      year_str = f"{date.year:04d}"
      month_str = f"{date.month:02d}"
      day_str = f"{date.day:02d}"

      # Format the hour with leading zeros
      hour_str = f"{int(hour):02d}"
    else:
      # Extract year, month, day, and format them with leading zeros
      year_str = f"{date_str.year:04d}"
      month_str = f"{date_str.month:02d}"
      day_str = f"{date_str.day:02d}"
      hour_str = f"{date_str.hour:02d}"

    return year_str, month_str, day_str, hour_str

print("Now figuring out what to do based on your inputs...")
if not Composite_Sounding:
  #checking for start time > end time
  if fcst_start > fcst_end:
    print(f"Your start hour is after your end hour!  Will fix this for you but you will only get forecast hour {fcst_start}")
    fcst_start = fcst_end


  # Adjusting time for RAOB synoptic times
  if source == "Obs":
    print("Checking to make sure your date/time matches synoptic hours...")
    date, hour = adjust_time(date, hour)
    print(f"Your date is {date} and your hour is {hour}")

  timelist = list(range(fcst_start, fcst_end+1))
  print(f"Your forecast hours are {timelist}")
  print("This will not apply to RAOB data ")

  # formating dates
  year, month, day, hour = format_date_and_hour(date, hour)

  ##################### Accessing Data #######################################

  if source == "Bufkit":
    print(f"Downloading Bufkit Data for the {year}{month}{day} {hour}Z run of the {model}...")
    for fcst_hr in timelist:
      print(f"Downloading forecast hour {fcst_hr}...")
      sourcefile = f"{model}_{year}{month}{day}_{hour}Z_F{fcst_hr:02d}"
      try:
        clean_data = spy.get_bufkit_data(model, site, fcst_hr, year, month, day, hour)
        spy.build_sounding(clean_data, style=style, dark_mode=Dark_Mode_Plot, storm_motion=Storm_Motion, show_radar=False, modify_sfc=Modified_T_Td, save=True, filename=f"{sourcefile}.png")
        print(f"Saved Bufkit sounding for {site} for F{fcst_hr:02d}")
      except:
        print(f"No data available for {model} at {site} for forecast hour {fcst_hr}")

  elif source == "Obs":
    print(f"Downloading RAOB Data for the {year}{month}{day} {hour}Z...")
    sourcefile = f"{ob_source}_{year}{month}{day}_{hour}Z"
    try:
      clean_data = spy.get_obs_data(site, year, month, day, hour)
      # first here we can make a sounding!
      spy.build_sounding(clean_data, style=style, dark_mode=Dark_Mode_Plot, storm_motion=Storm_Motion, show_radar=False, modify_sfc=Modified_T_Td, save=True, filename=f"{sourcefile}.png")
      print(f"Saved RAOB sounding for {site}")
    except Exception as e:
      print(e)
      print(f"No RAOB data available for {site} at {year}{month}{day} {hour}Z")

else:
  print("Composite soundings are a work in progress.  There may be bugs!")
  date_list = list(dates.split(","))
  model_list = list(models.upper().split(","))
  hour_list = [int(x) for x in list(hours.split(","))]
  #print(date_list)
  #print(model_list)
  #print(hour_list)
  composite_list = []
  for dt in date_list:
    #checking for RAOB
    if "RAOB" in model_list:
      print("Checking to make sure your date/time matches synoptic hours...")
      try:
        date, hour = adjust_time_composite(dt)
        raob_year, raob_month, raob_day, raob_hour = format_date_and_hour(date, hour)
      except ValueError:
        print("Your date/time format is wrong!  Make sure you have it in the format YYYY-mm-dd-hh")
      print(f"Your date is {date} and your hour is {hour}")
      for model in model_list:
        print(f"Now working on {model}")
        if model != "RAOB":
          for hr in hour_list:
            print(f"Downloading forecast hour {hr}...")
            initial_time = datetime.strptime(dt, "%Y-%m-%d-%H") - timedelta(hours=hr)
            if initial_time.hour in model_run_hours:
              year, month, day, hour = format_date_and_hour(initial_time, hour, composite=True)
              print(f"Year is: {year} and month is {month} and day is {day} and hour is {hour}")
              try:
                model_data = spy.get_bufkit_data(model, site, hr, year, month, day, hour, hush=True)
                composite_list.append(model_data)
              except:
                print(f"No data available for {model} at {site} for forecast hour {hr}")
            else:
              print(f'Initialization time of {initial_time.hour:02d}Z not valid for {model}.  Will have to skip this forecast hour')
              continue
        else:
          raob_data = spy.get_obs_data(site, raob_year, raob_month, raob_day, raob_hour, hush=True)
          composite_list.append(raob_data)
  #Building our sounding composite with the data we have gathered thus far
  print("Now building our composite sounding...")
  if len(date_list) == 1:
    fldate, flhour = adjust_time_composite(date_list[0])
    flyear, flmonth, flday, flhour = format_date_and_hour(fldate, flhour)
    sourcefile = f"Composite_{composite_site}_{flyear}{flmonth}{flday}_{flhour}Z"
  else:
    fldate, flhour = adjust_time_composite(date_list[0])
    flyear, flmonth, flday, flhour = format_date_and_hour(fldate, flhour)
    enddate, endhour = adjust_time_composite(date_list[-1])
    endyear, endmonth, endday, endhour = format_date_and_hour(enddate, endhour)
    sourcefile = f"Composite_{composite_site}_{flyear}{flmonth}{flday}_{flhour}Z_{enddate}{endmonth}{endday}_{endhour}Z"

  spy.build_composite(composite_list, dark_mode=Dark_Mode_Composite, save=True, filename=f"{sourcefile}.png")